In [12]:
%load_ext autoreload
%autoreload 2

import sys
import os
from datetime import datetime, timedelta
import pandas as pd

# Add project root to Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import src.config as config
from src.inference import get_feature_store, load_model_from_registry, get_model_predictions
from src.data_utils import transform_ts_data_info_features


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
%load_ext autoreload
%autoreload 2

import sys
import os
from datetime import datetime, timedelta
import pandas as pd

# Add project root to Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import src.config as config
from src.inference import get_feature_store, load_model_from_registry, get_model_predictions
from src.data_utils import transform_ts_data_info_features


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# Load feature store and feature view
feature_store = get_feature_store()
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

# Get batch of data around the time window (+/- 1 day for safety)
ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)

# Filter strictly for the target range
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data = ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

# Drop timezone if present
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)


2025-05-03 00:47:55,407 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-03 00:47:55,412 INFO: Initializing external client
2025-05-03 00:47:55,412 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-03 00:47:56,084 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214683
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.68s) 


In [15]:
# Create time-series features with sliding window
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)
print("✅ Features ready for prediction:", features.shape)


Skipping location_id 99: Not enough data to create even one window.
✅ Features ready for prediction: (510, 674)


In [16]:
# Load trained model from registry
model = load_model_from_registry()

# Get predictions
predictions = get_model_predictions(model, features)
print("✅ Predictions generated:", predictions.shape)
predictions.head()


2025-05-03 00:48:10,152 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-03 00:48:10,157 INFO: Initializing external client
2025-05-03 00:48:10,157 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-03 00:48:10,679 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214683


Downloading: 0.000%|          | 0/316530 elapsed<00:00 remaining<?

✅ Predictions generated: (510, 2)s, 1 files)... DONE


,pickup_location_id,predicted_demand
0,2,0.0
1,2,0.0
2,3,0.0
3,3,0.0
4,4,3.0


In [18]:
from src.inference import get_model_predictions

# Step 1: Get predictions
predictions = get_model_predictions(model, features)

# ✅ Step 2: Add pickup_hour back from features
predictions["pickup_hour"] = features["pickup_hour"].values

# Step 3: View top 10 high-demand predictions
top_locations = predictions.sort_values("predicted_demand", ascending=False)
top_locations[["pickup_location_id", "pickup_hour", "predicted_demand"]].head(10)


,pickup_location_id,pickup_hour,predicted_demand
307,161,2025-03-31 22:00:00,339.0
248,132,2025-03-30 23:00:00,301.0
249,132,2025-03-31 22:00:00,278.0
309,162,2025-03-31 22:00:00,267.0
443,230,2025-03-31 22:00:00,250.0
455,236,2025-03-31 22:00:00,231.0
269,142,2025-03-31 22:00:00,227.0
311,163,2025-03-31 22:00:00,225.0
451,234,2025-03-31 22:00:00,221.0
260,138,2025-03-30 23:00:00,217.0
